In [1]:
# Part 1 : gensim LDA based on NLTK & SpaCy

# Run in python console
import nltk; nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ipekcinar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# Turkish stop words
stop_words.extend(['a', 'acaba', 'altı', 'altmış', 'ama', 'ancak', 'arada', 'artık', 'asla', 'aslında', 'aslında', 'ayrıca', 'az', 'bana', 'bazen', 'bazı', 'bazıları', 'belki', 'ben', 'benden', 'beni', 'benim', 'beri', 'beş', 'bile', 'bilhassa', 'bin', 'bir', 'biraz', 'birçoğu', 'birçok', 'biri', 'birisi', 'birkaç', 'birşey', 'biz', 'bizden', 'bize', 'bizi', 'bizim', 'böyle', 'böylece', 'bu', 'buna', 'bunda', 'bundan', 'bunlar', 'bunları', 'bunların', 'bunu', 'bunun', 'burada', 'bütün', 'çoğu', 'çoğunu', 'çok', 'çünkü', 'da', 'daha', 'dahi', 'dan', 'de', 'defa', 'değil', 'diğer', 'diğeri', 'diğerleri', 'diye', 'doksan', 'dokuz', 'dolayı', 'dolayısıyla', 'dört', 'e', 'edecek', 'eden', 'ederek', 'edilecek', 'ediliyor', 'edilmesi', 'ediyor', 'eğer', 'elbette', 'elli', 'en', 'etmesi', 'etti', 'ettiği', 'ettiğini', 'fakat', 'falan', 'filan', 'gene', 'gereği', 'gerek', 'gibi', 'göre', 'hala', 'halde', 'halen', 'hangi', 'hangisi', 'hani', 'hatta', 'hem', 'henüz', 'hep', 'hepsi', 'her', 'herhangi', 'herkes', 'herkese', 'herkesi', 'herkesin', 'hiç', 'hiçbir', 'hiçbiri', 'i', 'ı', 'için', 'içinde', 'iki', 'ile', 'ilgili', 'ise', 'işte', 'itibaren', 'itibariyle', 'kaç', 'kadar', 'karşın', 'kendi', 'kendilerine', 'kendine', 'kendini', 'kendisi', 'kendisine', 'kendisini', 'kez', 'ki', 'kim', 'kime', 'kimi', 'kimin', 'kimisi', 'kimse', 'kırk', 'madem', 'mi', 'mı', 'milyar', 'milyon', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nedenle', 'nerde', 'nerede', 'nereye', 'neyse', 'niçin', 'nin', 'nın', 'niye', 'nun', 'nün', 'o', 'öbür', 'olan', 'olarak', 'oldu', 'olduğu', 'olduğunu', 'olduklarını', 'olmadı', 'olmadığı', 'olmak', 'olması', 'olmayan', 'olmaz', 'olsa', 'olsun', 'olup', 'olur', 'olur', 'olursa', 'oluyor', 'on', 'ön', 'ona', 'önce', 'ondan', 'onlar', 'onlara', 'onlardan', 'onları', 'onların', 'onu', 'onun', 'orada', 'öte', 'ötürü', 'otuz', 'öyle', 'oysa', 'pek', 'rağmen', 'sana', 'sanki', 'sanki', 'şayet', 'şekilde', 'sekiz', 'seksen', 'sen', 'senden', 'seni', 'senin', 'şey', 'şeyden', 'şeye', 'şeyi', 'şeyler', 'şimdi', 'siz', 'siz', 'sizden', 'sizden', 'size', 'sizi', 'sizi', 'sizin', 'sizin', 'sonra', 'şöyle', 'şu', 'şuna', 'şunları', 'şunu', 'ta', 'tabii', 'tam', 'tamam', 'tamamen', 'tarafından', 'trilyon', 'tüm', 'tümü', 'u', 'ü', 'üç', 'un', 'ün', 'üzere', 'var', 'vardı', 've', 'veya', 'ya', 'yani', 'yapacak', 'yapılan', 'yapılması', 'yapıyor', 'yapmak', 'yaptı', 'yaptığı', 'yaptığını', 'yaptıkları', 'ye', 'yedi', 'yerine', 'yetmiş', 'yi', 'yı', 'yine', 'yirmi', 'yoksa', 'yu', 'yüz', 'zaten', 'zira'
                  "acaba","acep","adeta","altmýþ","altmış","altý","altı","ama","ancak","arada","artýk","aslında","aynen","ayrıca","az","bana","bari","bazen","bazý","bazı","baţka","belki","ben","benden","beni","benim","beri","beþ","beş","beţ","bile","bin","bir","biraz","biri","birkaç","birkez","birçok","birþey","birþeyi","birşey","birşeyi","birţey","biz","bizden","bize","bizi","bizim","bu","buna","bunda","bundan","bunlar","bunları","bunların","bunu","bunun","burada","böyle","böylece","bütün","da","daha","dahi","dahil","daima","dair","dayanarak","de","defa","deđil","değil","diye","diđer","diğer","doksan","dokuz","dolayı","dolayısıyla","dört","edecek","eden","ederek","edilecek","ediliyor","edilmesi","ediyor","elli","en","etmesi","etti","ettiği","ettiğini","eđer","eğer","fakat","gibi","göre","halbuki","halen","hangi","hani","hariç","hatta","hele","hem","henüz","hep","hepsi","her","herhangi","herkes","herkesin","hiç","hiçbir","iken","iki","ila","ile","ilgili","ilk","illa","ise","itibaren","itibariyle","iyi","iyice","için","işte","iţte","kadar","kanýmca","karşın","katrilyon","kendi","kendilerine","kendini","kendisi","kendisine","kendisini","kere","kez","keţke","ki","kim","kimden","kime","kimi","kimse","kýrk","kýsaca","kırk","lakin","madem","međer","milyar","milyon","mu","mü","mý","mı","nasýl","nasıl","ne","neden","nedenle","nerde","nere","nerede","nereye","nitekim","niye","niçin","o","olan","olarak","oldu","olduklarını","olduğu","olduğunu","olmadı","olmadığı","olmak","olması","olmayan","olmaz","olsa","olsun","olup","olur","olursa","oluyor","on","ona","ondan","onlar","onlardan","onlari","onlarýn","onları","onların","onu","onun","otuz","oysa","pek","rağmen","sadece","sanki","sekiz","seksen","sen","senden","seni","senin","siz","sizden","sizi","sizin","sonra","tarafından","trilyon","tüm","var","vardı","ve","veya","veyahut","ya","yahut","yani","yapacak","yapmak","yaptı","yaptıkları","yaptığı","yaptığını","yapılan","yapılması","yapıyor","yedi","yerine","yetmiþ","yetmiş","yetmiţ","yine","yirmi","yoksa","yüz","zaten","çok","çünkü","öyle","üzere","üç","þey","þeyden","þeyi","þeyler","þu","þuna","þunda","þundan","þunu","şey","şeyden","şeyi","şeyler","şu","şuna","şunda","şundan","şunları","şunu","şöyle","ţayet","ţimdi","ţu","ţöyle"])  # make, come, go also very common


In [4]:
# Import Dataset -- original source
'''
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])
'''

'\ndf = pd.read_json(\'https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json\')\nprint(df.target_names.unique())\ndf.head()\n\n# Convert to list\ndata = df.content.values.tolist()\n\n# Remove Emails\ndata = [re.sub(\'\\S*@\\S*\\s?\', \'\', sent) for sent in data]\n\n# Remove new line characters\ndata = [re.sub(\'\\s+\', \' \', sent) for sent in data]\n\n# Remove distracting single quotes\ndata = [re.sub("\'", "", sent) for sent in data]\n\npprint(data[:1])\n'

In [5]:
# Import Dataset -- Shakespeare

# Pull all into "data" = all Shakespeare raw text

# directory containing all source texts for training the model 
data_dir="/Users/ipekcinar/Desktop/populism-hackathon/corpus_populism/Turkey"
import glob, os
os.chdir(data_dir)

#documents = list()
data = list()   # reset data to 0

for filename in glob.glob("*.txt"):
    filedata = open(filename, 'r').read()
    print(filename + " = " + str(len(filedata)) + " chars")
    #documents = documents + filedata.split(".")
    data.append(filedata)


Tayyip_2011_Nevsehir.txt = 36346 chars
Tayyip_2011_Mersin.txt = 29445 chars
Tayyip_2011_Agustos_Ulus.txt = 21624 chars
Tayyip_2011_Elazig.txt = 20565 chars
Tayyip_2011_Van.txt = 37228 chars
Tayyip_2011_Eskisehir.txt = 23219 chars
Tayyip_2011_Trabzon.txt = 38226 chars
Tayyip_2010_Ulus.txt = 20055 chars
Tayyip_2011_Adiyaman.txt = 32340 chars
Tayyip_2011_Agri.txt = 30624 chars
Tayyip_2011_Mardin.txt = 33328 chars
Tayyip_2011_Sakarya.txt = 26858 chars
Tayyip_2011_Ankara.txt = 36888 chars
Tayyip_2011_Isparta.txt = 31687 chars
Tayyip_2011_Diyarbakir.txt = 40110 chars
Tayyip_2011_Sirnak.txt = 30510 chars
Tayyip_2011_Balkon.txt = 11610 chars
Tayyip_2011_Izmir.txt = 44886 chars
Tayyip_2011_Aksaray.txt = 24827 chars
Tayyip_2011_Bursa.txt = 33978 chars
Tayyip_2011_Ocak.txt = 22737 chars
Tayyip_2011_Maltepe.txt = 29405 chars
Tayyip_2011_Mugla.txt = 36030 chars
Tayyip_2011_Konya.txt = 41272 chars
Tayyip_Ulus_2011.txt = 24315 chars
Tayyip_2011_Adana.txt = 32037 chars
Tayyip_2011_Hatay.txt = 35512 ch

In [6]:
%%time

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])
print('\n')

[[u'tum', u'nevsehirli', u'kardeslerimi', u'sevgiyle', u'sayg\u0131yla', u'en', u'kalbi', u'duygular\u0131mla', u'selaml\u0131yorum', u'buradan', u'ac\u0131gol', u'avanos', u'gulsehir', u'hac\u0131bektas', u'kozakl\u0131', u'ya', u'urgup', u'oralarda', u'yasayan', u'tum', u'kardeslerime', u'selamlar\u0131m\u0131', u'sevgilerimi', u'yolluyorum', u'eylul', u'halkoylamas\u0131nda', u'yuzde', u'gibi', u'yuksek', u'bir', u'oy', u'oran\u0131yla', u'demokrasiye', u'ozgurluklere', u'hukukun', u'ustunlugune', u'sahip', u'c\u0131kan', u'cetelere', u'dur', u'diyen', u'nevsehir', u'sukranlar\u0131m\u0131', u'sunuyorum', u'sizden', u'ald\u0131g\u0131m\u0131z', u'gucle', u'sizden', u'ald\u0131g\u0131m\u0131z', u'yetkiyle', u'sizlerin', u'hay\u0131r', u'duas\u0131yla', u'bu', u'yola', u'c\u0131kt\u0131k', u'bu', u'yolda', u'ilerliyoruz', u'sizin', u'cizdiginiz', u'istikamette', u'sizin', u'rotan\u0131zda', u'yuruyoruz', u'ak', u'parti', u'yi', u'millet', u'kurmustur', u'ak', u'parti', u'nin', u'kilim

In [7]:
#%%time

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])
print('\n')

[u'tum', u'nevsehirli', u'kardeslerimi', u'sevgiyle_sayg\u0131yla', u'en_kalbi_duygular\u0131mla_selaml\u0131yorum', u'buradan', u'ac\u0131gol', u'avanos', u'gulsehir', u'hac\u0131bektas', u'kozakl\u0131', u'ya', u'urgup', u'oralarda_yasayan_tum', u'kardeslerime_selamlar\u0131m\u0131_sevgilerimi_yolluyorum', u'eylul', u'halkoylamas\u0131nda', u'yuzde', u'gibi', u'yuksek', u'bir', u'oy', u'oran\u0131yla', u'demokrasiye', u'ozgurluklere', u'hukukun', u'ustunlugune', u'sahip', u'c\u0131kan', u'cetelere', u'dur', u'diyen', u'nevsehir', u'sukranlar\u0131m\u0131_sunuyorum', u'sizden', u'ald\u0131g\u0131m\u0131z', u'gucle', u'sizden', u'ald\u0131g\u0131m\u0131z', u'yetkiyle', u'sizlerin', u'hay\u0131r', u'duas\u0131yla', u'bu', u'yola_c\u0131kt\u0131k', u'bu', u'yolda', u'ilerliyoruz', u'sizin', u'cizdiginiz', u'istikamette', u'sizin', u'rotan\u0131zda', u'yuruyoruz', u'ak_parti', u'yi', u'millet', u'kurmustur', u'ak_parti', u'nin', u'kilimini', u'millet', u'dokumustur', u'ak_parti', u'nin', 

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
%%time
# This cell takes 2-3 minutes to run on my machine.  -j

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])
print('\n')

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  This is separate from the ipykernel package so we can avoid doing imports until


[[u'tum', u'nevsehirli', u'kardeslerimi', u'sevgiyle', u'sayg', u'kalbi', u'duygular', u'mla', u'selaml', u'yorum', u'buradan', u'gol', u'avano', u'gulsehir', u'hac', u'bekta', u'kozakl', u'urgup', u'oralarda_yasayan', u'tum', u'kardeslerime', u'selamlar', u'sevgilerimi_yolluyorum', u'eylul', u'halkoylama', u'nda', u'yuzde', u'yuksek', u'oran', u'yla', u'demokrasiye', u'ozgurluklere', u'hukukun', u'ustunlugune', u'sahip', u'kan', u'cetelere', u'dur', u'diyen', u'nevsehir', u'sukranlar', u'sunuyorum', u'ald', u'gucle', u'ald', u'yetkiyle', u'sizlerin', u'hay', u'duas', u'yla', u'yola', u'kt', u'yolda', u'ilerliyoruz', u'cizdiginiz', u'istikamette', u'rotan', u'zda', u'yuruyoruz', u'ak_parti', u'millet', u'kurmustur', u'kilimini', u'millet', u'dokumustur', u'hamurunu', u'millet', u'yogurmustur', u'baska', u'milletten', u'baska', u'hic', u'kimseden', u'talimat', u'almay', u'baska', u'kimsenin', u'onunde', u'boyun', u'egmeyiz', u'onurunuz', u'onurumuzdur', u'davan', u'davam', u'itibar', u'

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 1), (4, 1), (5, 3), (6, 8), (7, 2), (8, 2), (9, 2), (10, 2), (11, 4), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 2), (22, 2), (23, 2), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 9), (35, 1), (36, 6), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 2), (45, 1), (46, 2), (47, 3), (48, 3), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 3), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 2), (69, 3), (70, 1), (71, 2), (72, 1), (73, 1), (74, 3), (75, 1), (76, 2), (77, 2), (78, 1), (79, 1), (80, 1), (81, 13), (82, 2), (83, 1), (84, 1), (85, 1), (86, 3), (87, 1), (88, 1), (89, 1), (90, 1), (91, 2), (92, 2), (93, 1), (94, 8), (95, 5), (96, 2), (97, 5), (98, 1), (99, 3), (100, 1), (101, 1), (102, 7), (103, 32), (104, 2), (105, 1), (106, 4), (107, 1), (108, 8), (109, 2), (110, 

In [11]:
id2word[0]

u'abaza'

In [12]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(u'abaza', 1),
  (u'abla', 3),
  (u'ac', 1),
  (u'acan', 1),
  (u'acanlara', 1),
  (u'act', 3),
  (u'ad', 8),
  (u'adab', 2),
  (u'adam', 2),
  (u'adem', 2),
  (u'ademligi', 2),
  (u'adet', 4),
  (u'adet_aile', 1),
  (u'aff', 1),
  (u'affedecegini', 1),
  (u'agz', 1),
  (u'ahdimizi', 1),
  (u'ahi', 1),
  (u'ahmet', 1),
  (u'aile_doktoru', 1),
  (u'ak', 2),
  (u'ak_parti', 2),
  (u'akdeniz', 2),
  (u'akl', 2),
  (u'aksam', 1),
  (u'aksaray', 1),
  (u'aktaracag', 2),
  (u'aktif', 1),
  (u'ala', 1),
  (u'alacag', 1),
  (u'alam', 1),
  (u'alamam', 1),
  (u'alan', 1),
  (u'alanlar', 1),
  (u'ald', 9),
  (u'aldatmay', 1),
  (u'allah', 6),
  (u'allah_tan', 1),
  (u'almad', 1),
  (u'almanya', 1),
  (u'almay', 1),
  (u'alt', 1),
  (u'ambulan', 2),
  (u'ambulansla', 1),
  (u'ambulanslar', 2),
  (u'ameliyat', 1),
  (u'amerika', 2),
  (u'ana', 3),
  (u'anadolu', 3),
  (u'analar', 1),
  (u'anap', 1),
  (u'anapara', 2),
  (u'anayasan', 1),
  (u'anayasay', 1),
  (u'ankara', 1),
  (u'anl', 1),
  (u'

In [13]:
%%time

### Much as we saw with Word2vec, these settings are the key to tuning your LDA Topic Model. ###

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

CPU times: user 11.9 s, sys: 84.7 ms, total: 12 s
Wall time: 6.22 s


In [14]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  u'0.013*"kardeslerim" + 0.011*"simdi" + 0.008*"yor" + 0.007*"bak" + 0.007*"icin" + 0.006*"chp" + 0.006*"yapt" + 0.006*"yap" + 0.006*"nda" + 0.006*"degerli"'),
 (1,
  u'0.009*"nda" + 0.008*"turkiye" + 0.005*"icin" + 0.004*"eskisehir" + 0.004*"cok" + 0.004*"buyuk" + 0.004*"rnak" + 0.003*"butun" + 0.003*"ndan" + 0.003*"bugun"'),
 (2,
  u'0.000*"kardeslerim" + 0.000*"yapt" + 0.000*"simdi" + 0.000*"nda" + 0.000*"yor" + 0.000*"cok" + 0.000*"bak" + 0.000*"icin" + 0.000*"turkiye" + 0.000*"degerli"'),
 (3,
  u'0.010*"simdi" + 0.009*"kardeslerim" + 0.007*"yor" + 0.006*"nda" + 0.006*"chp" + 0.006*"icin" + 0.005*"cok" + 0.005*"yapt" + 0.005*"bak" + 0.005*"yap"'),
 (4,
  u'0.011*"kardeslerim" + 0.010*"simdi" + 0.008*"yor" + 0.007*"icin" + 0.007*"nda" + 0.007*"yapt" + 0.006*"bak" + 0.006*"chp" + 0.005*"yap" + 0.005*"turkiye"'),
 (5,
  u'0.011*"kardeslerim" + 0.011*"simdi" + 0.008*"bak" + 0.008*"nda" + 0.007*"trabzon" + 0.006*"degerli" + 0.006*"yapt" + 0.006*"yor" + 0.005*"icin" + 0.005*"degil

In [15]:
%%time

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print('\n')

('\nPerplexity: ', -8.458841221258922)
('\nCoherence Score: ', 0.26551037855591186)


CPU times: user 1.45 s, sys: 62.4 ms, total: 1.51 s
Wall time: 6.1 s


In [16]:
%%time

# Visualize the topics
#
# If you get an error like this: "pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. 
# A future version of pandas will change to not sort by default."
#
# then from the command line do: "pip install pandas==0.21.0"

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)  # sort=False ? sort=True
vis
print('\n')



CPU times: user 793 ms, sys: 137 ms, total: 930 ms
Wall time: 11.8 s


/anaconda2/lib/python2.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


#import pandas as pd
#pd.__version__

vis

In [17]:
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      40.577816        1       1  0.067397  0.013089
0      28.075283        1       2  0.089697  0.006896
3      16.061283        1       3  0.061375 -0.007033
1       8.212893        1       4 -0.096406  0.112913
5       7.072587        1       5  0.034253 -0.051730
2       0.000133        1       6 -0.156315 -0.074135, topic_info=      Category         Freq             Term        Total  loglift  logprob
term                                                                       
6348   Default  1302.000000      kardeslerim  1302.000000  30.0000  30.0000
6308   Default  1154.000000            simdi  1154.000000  29.0000  29.0000
17919  Default   833.000000              nda   833.000000  28.0000  28.0000
9689   Default   722.000000              bak   722.000000  27.0000  27.0000
1520   Default   570.000000          turkiye   570.000000  26.0000  26.0000
6679   Default   569.000000          degerli   569.000000  25.0000  25.0000
13532  Default   731.000000             yapt   731.000000  24.0000  24.0000
10590  Default    79.000000          trabzon    79.000000  23.0000  23.0000
13367  Default   794.000000             icin   794.000000  22.0000  22.0000
5116   Default   491.000000            diyor   491.000000  21.0000  21.0000
5176   Default   573.000000              cok   573.000000  20.0000  20.0000
6749   Default   896.000000              yor   896.000000  19.0000  19.0000
5391   Default   680.000000              chp   680.000000  18.0000  18.0000
17386  Default   439.000000  biliyor_musunuz   439.000000  17.0000  17.0000
4908   Default   539.000000            degil   539.000000  16.0000  16.0000
9465   Default    64.000000        eskisehir    64.000000  15.0000  15.0000
6741   Default   410.000000              yok   410.000000  14.0000  14.0000
2195   Default   250.000000               ad   250.000000  13.0000  13.0000
7107   Default   386.000000             lira   386.000000  12.0000  12.0000
7347   Default   325.000000              kar   325.000000  11.0000  11.0000
6772   Default   301.000000            bugun   301.000000  10.0000  10.0000
4278   Default   318.000000              mhp   318.000000   9.0000   9.0000
13633  Default   253.000000            buyuk   253.000000   8.0000   8.0000
1565   Default   204.000000          bahceli   204.000000   7.0000   7.0000
4751   Default   511.000000              lar   511.000000   6.0000   6.0000
1637   Default   349.000000              say   349.000000   5.0000   5.0000
14050  Default   366.000000            yuzde   366.000000   4.0000   4.0000
2912   Default    67.000000           samsun    67.000000   3.0000   3.0000
12765  Default   300.000000            butun   300.000000   2.0000   2.0000
6539   Default   472.000000          sevgili   472.000000   1.0000   1.0000
...        ...          ...              ...          ...      ...      ...
17919   Topic6     0.000047              nda   833.052429  -3.1656  -8.1540
13956   Topic6     0.000009      gorebiliyor     1.458138   1.5085  -9.8278
5176    Topic6     0.000041              cok   573.618469  -2.9203  -8.2818
6749    Topic6     0.000045              yor   896.385620  -3.2872  -8.2023
15011   Topic6     0.000009         surecten     1.459459   1.5078  -9.8276
1520    Topic6     0.000038          turkiye   570.357605  -2.9887  -8.3559
12982   Topic6     0.000035             iste   428.495392  -2.7840  -8.4372
6679    Topic6     0.000037          degerli   569.971130  -3.0082  -8.3760
9689    Topic6     0.000039              bak   722.927002  -3.2042  -8.3344
4908    Topic6     0.000036            degil   539.462952  -2.9963  -8.4192
6539    Topic6     0.000034          sevgili   472.272430  -2.9056  -8.4615
13367   Topic6     0.000038             icin   794.294922  -3.3140  -8.3500
4751    Topic6     0.000035              lar   511.884247  -2.9759  -8.4513
5391    Topic6     0.000036  